In [1]:
%config Completer.use_jedi = False
import warnings
warnings.filterwarnings(action="ignore")

In [2]:
# seleniumwire가 import가 안되면 python -m pip install --upgrade pip를 실행하고 다시 설치한다.
from seleniumwire import webdriver
import json
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

seleniumwire를 이용한 스타벅스 시도코드 크롤링

In [3]:
options = Options()
options.add_argument('--window-size= 1024, 1680')
driver = webdriver.Chrome(options = options)
driver.implicitly_wait(5)
#get() 메소드로 XHR/Fetch 요청이 들어가는 웹페이지를 요청한다.
driver.get('https://www.starbucks.co.kr:7643/store/store_map.do?disp=locale')

#wait_for_request() 메소드로 XHR/Fetch 요청이 들어가는 웹페이지가 응답하는 결과를 받는다.
request = driver.wait_for_request('https://www.starbucks.co.kr:7643/store/getSidoList.do', timeout=10)
#print(request.url)
#print(request.response.status_code)
#print(request.response.headers)
#print(request.response.headers['content-type'])
#print('body')
#print(request.response.body.decode('utf-8'))
#print(type(request.response.body.decode('utf-8')))

starbucks = json.loads(request.response.body.decode('utf-8'))
#print(type(starbucks))
#print(starbucks)

sido_list = {}
for star in starbucks['list']:
    sido_list[star.get('sido_nm')] = star['sido_cd']
#------- for star
print(sido_list)

The chromedriver version (123.0.6312.122) detected in PATH at C:\Users\tj\Desktop\python\seleniumV4Crawling\chromedriver.exe might not be compatible with the detected chrome version (124.0.6367.61); currently, chromedriver 124.0.6367.60 is recommended for chrome 124.*, so it is advised to delete the driver in PATH and retry


{'서울': '01', '경기': '08', '광주': '02', '대구': '03', '대전': '04', '부산': '05', '울산': '06', '인천': '07', '강원': '09', '경남': '10', '경북': '11', '전남': '12', '전북': '13', '충남': '14', '충북': '15', '제주': '16', '세종': '17'}


seleniumwire를 이용한 스타벅스 구군코드 크롤링

In [4]:
# 서울 : //*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[1]/a  
# 경기 : //*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[2]/a  
# 광주 : //*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[3]/a  
# 대구 : //*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[4]/a  
#...   
# 세종 : //*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[17]/a  

In [5]:
options = Options()
options.add_argument('--window-size= 1920, 1680')
driver = webdriver.Chrome(options = options)
driver.implicitly_wait(5)
driver.get('https://www.starbucks.co.kr:7643/store/store_map.do?disp=locale')


sido_list = {
    
    '서울': '01', '경기': '02', '광주': '03', '대구': '04', '대전': '05', '부산': '06', '울산': '07', '인천': '08', '강원': '09',
    '경남': '10', '경북': '11', '전남': '12', '전북': '13', '충남': '14', '충북': '15', '제주': '16', '세종': '17'

}
sido = input('시도 입력 ??? -> ')


driver.find_element(By.XPATH, 
                    '//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li['
                        + str(int(sido_list[sido])) +']/a'
                    ).click()
time.sleep(1)
request = driver.wait_for_request('https://www.starbucks.co.kr:7643/store/getGugunList.do', timeout=10)
starbucks = json.loads(request.response.body.decode('utf-8'))
#print(starbucks)

gugun_list = {}
for i, star in zip(range(2, (len(starbucks['list']) + 2) ), starbucks['list']):
    #print(i, star['gugun_nm'])
    gugun_list[star['gugun_nm']] = i
#------- for star
print(gugun_list)

gugun= input('구군 -> ')
#driver.find_element(By.XPATH, 
#                    '/html/body/div[4]/div[7]/div/form/fieldset/div/section/article[1]/article/article[2]/div[2]/div[3]/div/div/div[1]/ul/li['+
#                        str(gugun_list[gugun]) +']/a'        
#                    ).click()
driver.find_element(By.CSS_SELECTOR, 
                    '#mCSB_2_container > ul > li:nth-child('+ str(gugun_list[gugun]) +') > a' 
                    ).click()
time.sleep(1)
request = driver.wait_for_request('https://www.starbucks.co.kr:7643/store/getStore.do', timeout=10)
starbucks = json.loads(request.response.body.decode('utf-8'))
store_list = {}
for i, star in zip( range (1, len(starbucks['list']) +1 ), starbucks['list'] ):
    print('{:3d}. {}: {}: <{},{}>'.format(i, star['s_name'], star['doro_address'], star['lat'], star['lot']))
    store_list[star['s_name']] = [star['doro_address'], star['lat'], star['lot']]
print(store_list)

The chromedriver version (123.0.6312.122) detected in PATH at C:\Users\tj\Desktop\python\seleniumV4Crawling\chromedriver.exe might not be compatible with the detected chrome version (124.0.6367.61); currently, chromedriver 124.0.6367.60 is recommended for chrome 124.*, so it is advised to delete the driver in PATH and retry


시도 입력 ??? -> 제주
{'서귀포시': 2, '제주시': 3}
구군 -> 제주시
  1. 제주평화로DT: 제주특별자치도 제주시 노형로 37 (해안동): <33.46329278001417,126.45099791468475>
  2. 더제주송당파크R: 제주특별자치도 제주시 구좌읍 비자림로 1189 : <33.4362649647325,126.731232423627>
  3. 제주공항DT: 제주특별자치도 제주시 월성로 42 (용담이동): <33.5005664,126.5054842>
  4. 제주일도DT: 제주특별자치도 제주시 동광로 130 (일도이동): <33.5073284,126.5408980>
  5. 제주서해안로DT: 제주특별자치도 제주시 서해안로 624 (용담삼동): <33.5167857,126.5035553>
  6. 제주한라수목원DT: 제주특별자치도 제주시 수목원서길 3-5 (노형동): <33.4690200090396,126.484585553868>
  7. 제주노형DT: 제주특별자치도 제주시 월랑로 68 (노형동): <33.4908499861488,126.474278374413>
  8. 제주노형로: 제주특별자치도 제주시 노형로 376 (노형동): <33.4834410103164,126.476159449097>
  9. 제주세화DT: 제주특별자치도 제주시 구좌읍 일주동로 3121 : <33.522979308914,126.8534175496>
 10. 신제주이마트: 제주특별자치도 제주시 1100로 3348, 신제주이마트 1층 (노형동): <33.484959,126.480478>
 11. 제주용담DT: 제주특별자치도 제주시 서해안로 380 (용담삼동) 화이트하우스: <33.512345,126.484564>
 12. 제주노형: 제주특별자치도 제주시 도령로 27 (노형동): <33.4878111,126.4818612>
 13. 제주칠성: 제주특별자치도 제주시 관덕로 55 (일도일동): <33.513349,126.5252435>
 14. 제주애월DT: 제주특

seleniumwire를 이용한 스타벅스 지역별 매장 정보 크롤링

In [6]:
#강남구 -> /html/body/div[4]/div[7]/div/form/fieldset/div/section/article[1]/article/article[2]/div[2]/div[3]/div/div/div[1]/ul/li[2]/a
#강동구 -> /html/body/div[4]/div[7]/div/form/fieldset/div/section/article[1]/article/article[2]/div[2]/div[3]/div/div/div[1]/ul/li[3]/a
#강북구 -> /html/body/div[4]/div[7]/div/form/fieldset/div/section/article[1]/article/article[2]/div[2]/div[3]/div/div/div[1]/ul/li[4]/a
